# Read BicikeLJ data

In [ ]:
# Matplotlib inline
%matplotlib inline
# Imports
from urllib.request import urlopen
import json

# import xml.etree.ElementTree as ET
import numpy as np
import pandas as pd
import datetime
import pytz

Branje podatkov po protokolih, opisanih v: https://developer.jcdecaux.com/#/opendata/vls?page=getstarted

Zanimiva je diplomska naloga: http://geo.ff.uni-lj.si/pisnadela/pdfs/zaksem_201409_ziga_jamnik.pdf

In [ ]:
# Parameters
# Station static infos
station_info_url = "https://developer.jcdecaux.com/rest/vls/stations/Ljubljana.json"
# Bycicle, station dynamic infos
station_data_url = "https://api.jcdecaux.com/vls/v1/stations?contract=Ljubljana&apiKey=0a494317d60d3d556d0755600b078ea6b26af90f"
# Station info dataframe
station_fn_ext = ".csv"
station_info_fn = "bicikelj_station_info" + station_fn_ext
station_data_fn = (
    "bicikelj_station_data_" + datetime.datetime.now().strftime("%y%m") + station_fn_ext
)

In [ ]:
station_data_fn

## Station info

In [ ]:
response = urlopen(station_data_url)
if response.code == 200:
    data = response.read().decode("utf-8")
else:
    print("Wrong response from ", station_data_url)

In [ ]:
response.code

In [ ]:
station_info_json = json.loads(data)

In [ ]:
station_info = (
    pd.DataFrame(station_info_json).sort_values(["number"]).reset_index(drop=True)
)

In [ ]:
station_info.columns

In [ ]:
station_info = station_info[
    [
        "address",
        "banking",
        "bike_stands",
        "bonus",
        "name",
        "number",
        "position",
        "status",
    ]
].copy()

In [ ]:
station_info = pd.concat(
    [
        station_info.drop(["position"], axis=1),
        station_info["position"].apply(pd.Series),
    ],
    axis=1,
)
station_info.head()

In [ ]:
# Save to CSV
station_info.to_csv(station_info_fn, index=True)

## Read station data

In [ ]:
response = urlopen(station_data_url)
if response.code == 200:
    data = response.read().decode("utf-8")
else:
    print("Wrong response from ", station_data_url)

In [ ]:
station_data_json = json.loads(data)

In [ ]:
station_data = (
    pd.DataFrame(station_data_json).sort_values(["number"]).reset_index(drop=True)
)

In [ ]:
station_data.columns

In [ ]:
station_data.head()

In [ ]:
station_data_real = station_data[
    ["available_bike_stands", "available_bikes", "bike_stands", "last_update", "number"]
].copy()

In [ ]:
station_data_real["last_update_time"] = pd.to_datetime(
    station_data_real["last_update"] * 1e6
)

In [ ]:
station_data_real = station_data_real.set_index(["last_update_time"])

In [ ]:
station_data_real.drop(["last_update"], 1).sort_index()

In [ ]:
local_tz = pytz.timezone("Europe/Ljubljana")
station_data_real.index = station_data_real.index.tz_localize(pytz.utc).tz_convert(
    local_tz
)

In [ ]:
station_data_real.head()

In [ ]:
station_data_full = pd.read_csv(
    station_data_fn, index_col="last_update_time", parse_dates=True
)

In [ ]:
station_data_full.head()

In [ ]:
type(station_data_full.index[0])

In [ ]:
added = station_data_full.append(station_data_real).drop_duplicates().sort_index()

In [ ]:
len(added)

## Analyze station data

In [ ]:
station_data_full = pd.read_csv(station_data_fn, index_col="last_update_time")

In [ ]:
station_data_full.describe()

In [ ]:
station_data_full[station_data_full["number"] == 3]["available_bike_stands"].plot(
    linestyle="steps"
)

In [ ]:
station_data_full[station_data_full["number"] == 3]["available_bikes"].plot(
    linestyle="steps"
)

# Convert Time Zone

In [ ]:
local_tz = pytz.timezone("Europe/Ljubljana")

In [ ]:
station_data_full = pd.read_csv(
    station_data_fn, index_col="last_update_time", parse_dates=True
)
station_data_full.index = (
    station_data_full.index.tz_localize(pytz.utc).tz_convert(local_tz).tz_localize(None)
)
station_data_full.to_csv(station_data_fn, index=True)

In [ ]:
meteo_data_fn = "meteo_data_1701.csv"
meteo_data_full = pd.read_csv(meteo_data_fn, index_col="DateTime", parse_dates=True)
meteo_data_full.index = (
    meteo_data_full.index.tz_localize(pytz.utc).tz_convert(local_tz).tz_localize(None)
)
meteo_data_full.to_csv(meteo_data_fn, index=True)